In [26]:
%pip install pandas dash

Note: you may need to restart the kernel to use updated packages.


In [22]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
from dash import Dash, dcc
#Create app
app = Dash(__name__)
#Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

In [28]:

  
import pandas as pd

spacex_df = pd.read_csv("spacex_launch_dash.csv")

from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Use existing spacex_df, min_value, max_value, and app

min_value = spacex_df['Payload Mass (kg)'].min()
max_value = spacex_df['Payload Mass (kg)'].max()

app.layout = html.Div(children=[
    html.H1("SpaceX Launch Records Dashboard",
        style={'textAlign': 'center','color': '#503D36', 'font-size': 24}),
    html.Div([
        dcc.Dropdown(
            id='site-dropdown',
            options=[
                {'label': 'All Sites', 'value': 'ALL'},
                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            ],
            value='ALL',
            placeholder="Select a Launch Site here",
            searchable=True
        ),
    html.Div([
            dcc.Graph(id='success-pie-chart')
            ]),
    html.Div([
        html.Label("Payload range (Kg):"),
        dcc.RangeSlider(
            id='payload-slider',
            min=0, max=10000, step=1000,
            marks={0: '0',
                   2500: '2500',
                   5000: '5000',
                   7500: '7500',
                   10000: '10000'},
            value=[min_value, max_value]
        )
    ], style={'padding': '20px 0px', 'color': '#503D36'}),
        
    html.Div([
            dcc.Graph(id='success-payload-scatter-chart')
            ]),
    ])
])

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches by Site')
    else:
        # Filter for the selected site
        site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        # Count success (1) and failure (0)
        site_counts = site_df['class'].value_counts().reset_index()
        site_counts.columns = ['Outcome', 'Count']
        site_counts['Outcome'] = site_counts['Outcome'].map({1: 'Success', 0: 'Failure'})
        
        fig = px.pie(site_counts, 
                     values='Count', 
                     names='Outcome', 
                     title=f'Total Success Launches for Site {entered_site}')
    return fig
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    filtered_df = spacex_df[mask]
    
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, 
                         x='Payload Mass (kg)', 
                         y='class', 
                         color='Booster Version Category',
                         title='Correlation between Payload and Success for all Sites')
    else:
        site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(site_df, 
                         x='Payload Mass (kg)', 
                         y='class', 
                         color='Booster Version Category',
                         title=f'Correlation between Payload and Success for site {entered_site}')
    return fig


# Run the app server (for Jupyter, use run)
app.run(debug=True, port=8051)